<a href="https://colab.research.google.com/github/Hectooooooor/AI/blob/main/4.0709_Colab_LINE_Bot_with_GEMINI_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 5.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [7]:
import requests
from pyngrok import ngrok

ngrok.kill() # Kill any existing ngrok tunnels
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token.strip()}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://liz-magnetostrictive-discomposingly.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://liz-magnetostrictive-discomposingly.ngrok-free.dev


True

In [8]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [9]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [10]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。學校創立於1966年，最初為明新工業專科學校，歷經1997年改制為明新技術學院，並於2002年獲教育部核准升格，正式定名為明新科技大學。2018年，更名為明新學校財團法人明新科技大學。

學校秉持「在明明德，在新民，在止於至善」的精義，以培育兼具高尚品德、專業學問與優良技術的專業人才為目標。明新科大的願景是成為「一流產業大學」，並期許學子具備專業技能、團隊精神、人文素養與宏觀服務的科技人才，致力於「深耕在地、放眼國際」.

明新科大現設有半導體學院、工程學院、管理學院、民生學院、人文與設計學院以及共同教育學院等六個學院，涵蓋20個學系、2個學位學程（含1個博士學位學程）及11個碩士班. 其中，半導體學院的畢業生在業界備受青睞，根據1111人力銀行統計，明新科大在半導體產業界最愛聘用的畢業生中名列前茅，與頂尖大學並列。2022年，明新科大更獲得《遠見》雜誌「企業最愛公私立技職科大調查」兩項企業最愛，起薪排名私校第一.

學校的辦學特色著重於「產業大學」的定位，與新竹科學園區及新竹工業區的產業資源緊密結合，強化產學鏈結，推動契合式人才課程，使畢業生具備「立即就業」的能力。明新科大積極推動「M.U.S.T.」四大育才特色，包括多元學習、全球視野、永續經營與技術創新，引導學生跨域學習，以符合半導體、AI、元宇宙、風電綠能等前瞻產業的需求。此外，學校也致力於建置「永續智慧商務」教學與實習場域，導入生成式AI與AI專案應用，發展智慧零售、智慧金融、智慧製造與智慧商業，並推動企業溫室氣體管理團隊及產業淨零碳排合作模式，以期學生與業界同步接軌。111年10月，獲教育部核准通過「半導體科技博士學位學程」，成為學校第一個博士班。


In [11]:
result2 = stateful_query("校長是誰？")
print(result2)

None


In [ ]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 儲存每個使用者的對話 session 和上傳的檔案
user_sessions = {}  # {user_id: {"chat": chat_object, "uploaded_file": gemini_file}}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:
        # 建立新的對話 session
        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    # 等待檔案處理完成
    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        # 有上傳檔案，使用 RAG 模式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        # 沒有上傳檔案，使用一般多輪對話
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 使用 RAG 或一般對話
                reply_text = query_with_rag(user_id, prompt)

                # 檢查是否有上傳檔案，加上提示
                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            # 清除使用者上傳的檔案
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 檢查檔案類型
        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            # 下載檔案
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            # 上傳到 Gemini
            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            # 儲存到使用者 session
            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[{"type":"message","message":{"type":"text","id":"595534223109259471","quoteToken":"dYJL_2Qg7GzO5PGvbffRTetD3kB-6YgkCDnmYnLH4cz9vNMW-BGjh9-vRgxIuVKhadPIYyXmSItYoGz-V_cJQlPKtli2HF6ljuSMKyEHEppdFvysbvy7e8diBA571KmYmawzHyqD63XASvxKoNCavQ","markAsReadToken":"V7dAvvE48RfP6X83e3vWujiqJLIE0SeJUbLZ0sFKnnHE0FXNyce41ocHz_g_QOyic12z6B74gGQYD6Evvh9DK8Tb7cHMX7OJp_n9V2k2YCmLH59wu3AMTpIgzCACUepjQ-TW-tRdJyzE4evwqSlif_kOmJi2Bkcft9DGhrUcA1bn66Rg8-dQw9TLYRIPzb09NL_7v3sLxeacnq4JfYnbpw","text":"AI 陳小明喜歡吃什麼？"},"webhookEventId":"01KECEY45F6K2WHM42T13PFQ1J","deliveryContext":{"isRedelivery":false},"timestamp":1767797362409,"source":{"type":"user","userId":"Ue020f480d3b30476c7126cc8cf6811c0"},"replyToken":"0ea1d30aef384985b

BODY:  {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[{"type":"message","message":{"type":"text","id":"595534223109259471","quoteToken":"dYJL_2Qg7GzO5PGvbffRTetD3kB-6YgkCDnmYnLH4cz9vNMW-BGjh9-vRgxIuVKhadPIYyXmSItYoGz-V_cJQlPKtli2HF6ljuSMKyEHEppdFvysbvy7e8diBA571KmYmawzHyqD63XASvxKoNCavQ","markAsReadToken":"V7dAvvE48RfP6X83e3vWujiqJLIE0SeJUbLZ0sFKnnHE0FXNyce41ocHz_g_QOyic12z6B74gGQYD6Evvh9DK8Tb7cHMX7OJp_n9V2k2YCmLH59wu3AMTpIgzCACUepjQ-TW-tRdJyzE4evwqSlif_kOmJi2Bkcft9DGhrUcA1bn66Rg8-dQw9TLYRIPzb09NL_7v3sLxeacnq4JfYnbpw","text":"AI 陳小明喜歡吃什麼？"},"webhookEventId":"01KECEY45F6K2WHM42T13PFQ1J","deliveryContext":{"isRedelivery":false},"timestamp":1767797362409,"source":{"type":"user","userId":"Ue020f480d3b30476c7126cc8cf6811c0"},"replyToken":"0ea1d30aef384985b62caedd96495199","mode":"active"}]}


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/tmp/ipython-input-1166925939.py", line 156, in handle_message
    line_bot_api.reply_message_with_http_info(
  File "/usr/local/lib/python3.12/dist-packages/pydantic/v1/decorator.py", line 40, in wrapper_function
    return vd.call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pydantic/v1/decorator.py", line 134, in call
    return self.execute(m)
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pydantic/v1/decorator.py", line 206, in execute
    return self.raw_function(**d, **var_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/linebot/v3/messaging/api/messaging_api.py", line 7666, in reply_message_with_http_info
    return self.api_client.call_api(
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/linebot/v3/messaging/api_client.py", l

BODY:  {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[{"type":"message","message":{"type":"text","id":"595534644536148160","quoteToken":"40iOx5sgxiU0A2tLzKrdLzZR0x35NI35Etzcay3f7578VM6_B8jOWgm10j5onUoPjIT83ahTPit72Y3jT6s7_kwICnXZl-IePhu_JRyjkNtQb2iY00Irhq0nW8JxHA8Uwi_DQWGfz4bDCVGK8zIAVQ","markAsReadToken":"9VNonoIDJaVK6Qp92hoRlF5iF_-VyA157mmla4hUykvkq_LP0OWK8qinupwk8Yb6WbvKuPzwAwqZoAa1p5zrjviWat4R-fGRoYoVoeGWL2TrhSs0jJuWXtp9ETmjysbof9UxHegGkzatSrpiTSk4fPU-ofqROMdtJOyYvTWzbeL6QW5ESPvOVT0XvU8x76_uzWNSX33BDIpae9EkKPNm5A","text":"AI 陳小明喜歡吃什麼？"},"webhookEventId":"01KECF5SFSZY3EX6WD1Z1CNWVS","deliveryContext":{"isRedelivery":false},"timestamp":1767797613566,"source":{"type":"user","userId":"Ue020f480d3b30476c7126cc8cf6811c0"},"replyToken":"2f344bc623a54d1c95cd272a0e27a28d","mode":"active"}]}


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/tmp/ipython-input-1166925939.py", line 156, in handle_message
    line_bot_api.reply_message_with_http_info(
  File "/usr/local/lib/python3.12/dist-packages/pydantic/v1/decorator.py", line 40, in wrapper_function
    return vd.call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pydantic/v1/decorator.py", line 134, in call
    return self.execute(m)
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pydantic/v1/decorator.py", line 206, in execute
    return self.raw_function(**d, **var_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/linebot/v3/messaging/api/messaging_api.py", line 7666, in reply_message_with_http_info
    return self.api_client.call_api(
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/linebot/v3/messaging/api_client.py", l